In [ ]:
import torch
import torchvision.transforms.functional as F
from PIL import Image
import matplotlib.pyplot as plt
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
# Set the path to the folder containing images in Google Drive
google_drive_folder_path = "/content/drive/MyDrive/HR"

# Set the path to the folder where you want to save the noisy images in Google Drive
output_folder_path = "/content/drive/MyDrive/NoisyHR"

# Ensure the output folder exists; create it if not
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# List all files in the input folder
image_files = os.listdir(google_drive_folder_path)

# Define the noise level (you can adjust this based on your preference)
noise_level = 0.1

# Loop through each image file in the folder
for image_file in image_files:
    # Load an image
    image_path = os.path.join(google_drive_folder_path, image_file)
    img = Image.open(image_path)

    # Convert the image to a PyTorch tensor
    img_tensor = F.to_tensor(img)

    # Generate Gaussian noise
    noise = torch.randn_like(img_tensor) * noise_level

    # Add noise to the image
    noisy_img = img_tensor + noise

    # Clip values to be in the valid range [0, 1]
    noisy_img = torch.clamp(noisy_img, 0, 1)

    # Convert the tensor back to a PIL image
    noisy_img_pil = F.to_pil_image(noisy_img)

    # Save the noisy image to the output folder
    output_path = os.path.join(output_folder_path, f"Noisy_{image_file}")
    noisy_img_pil.save(output_path)

    print(f"Noisy image saved: {output_path}")

# Display the first original and noisy images for visualization
# plt.subplot(1, 2, 1)
# plt.imshow(Image.open(os.path.join(google_drive_folder_path, image_files[0])))
# plt.title("Original Image")

# plt.subplot(1, 2, 2)
# plt.imshow(noisy_img_pil)
# plt.title("Noisy Image")

# plt.show()


In [ ]:
import os
from google.colab import drive
import torch
import torchvision.transforms.functional as F
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive

def add_salt_and_pepper_noise(image_tensor, noise_level=0.02):

    """
    Add salt-and-pepper noise to an input image tensor.

    Parameters:
    - image_tensor (torch.Tensor): Input image tensor.
    - noise_level (float): Intensity of the salt-and-pepper noise.

    Returns:
    - torch.Tensor: Noisy image tensor.

    """
    height, width = image_tensor.shape[1:]

    # Generate random positions for salt (white) pixels
    # salt_coords = torch.randint(0, 2, (height, width), device=image_tensor.device, dtype=torch.bool)

    salt_density = 0.02  # Adjust this value to control the density of salt pixels
    salt_coords = torch.rand((height, width), device=image_tensor.device) < salt_density
    salt_coords = salt_coords.to(torch.bool)

    # Generate random positions for pepper (black) pixels
    # pepper_coords = torch.randint(0, 2, (height, width), device=image_tensor.device, dtype=torch.bool)
    pepper_density = 0.02  # Adjust this value to control the density of salt pixels
    pepper_coords = torch.rand((height, width), device=image_tensor.device) < pepper_density
    pepper_coords = salt_coords.to(torch.bool)

    # Set salt pixels to 1.0 (white)
    image_tensor[:, salt_coords] = 1.0

    # Set pepper pixels to 0.0 (black)
    image_tensor[:, pepper_coords] = 0.0

    # Flip some of the salt and pepper pixels based on the noise level
    flip_mask = torch.rand_like(image_tensor) < noise_level
    image_tensor[flip_mask] = 1.0 - image_tensor[flip_mask]

    return image_tensor

# Mount Google Drive
drive.mount('/content/drive')

# Specify the paths for input and output folders
input_folder = "/content/drive/MyDrive/HR"  # Update with the path to your input images
output_folder = "/content/drive/MyDrive/N02"  # Update with the desired output path

# Create the output folder if it doesn't exist
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpeg") or filename.endswith(".png") or filename.endswith(".jpg"):  # Adjust file extensions as needed
        # Load an image
        image_path = os.path.join(input_folder, filename)
        img = Image.open(image_path)

        # Convert the image to a PyTorch tensor
        img_tensor = F.to_tensor(img)

        # Add salt-and-pepper noise
        noisy_img = add_salt_and_pepper_noise(img_tensor)

        # Convert the noisy tensor back to a PIL image
        noisy_img_pil = F.to_pil_image(noisy_img)

        # Save the noisy image to the output folder
        output_path = os.path.join(output_folder, f"Noisy_{filename}")
        noisy_img_pil.save(output_path)

# Print a message indicating the process is complete
print("Salt-and-pepper noise added to all images. Results saved in:", output_folder)
